除了庞大的数据集和强大的硬件， 优秀的软件工具在深度学习的快速发展中发挥了不可或缺的作用。 从2007年发布的开创性的Theano库开始， 灵活的开源工具使研究人员能够快速开发模型原型， 避免了我们使用标准组件时的重复工作， 同时仍然保持了我们进行底层修改的能力。 随着时间的推移，深度学习库已经演变成提供越来越粗糙的抽象。 就像半导体设计师从指定晶体管到逻辑电路再到编写代码一样， 神经网络研究人员已经从考虑**单个人工神经元**的行为转变为从**层**的角度构思网络， 通常在设计架构时考虑的是更粗糙的**块（block）**。

之前我们已经介绍了一些基本的机器学习概念， 并慢慢介绍了功能齐全的深度学习模型。 在上一章中，我们从零开始实现了多层感知机的每个组件， 然后展示了如何利用高级API轻松地实现相同的模型。 为了易于学习，我们调用了深度学习库，但是跳过了它们工作的细节。 在本章中，我们将深入探索深度学习计算的关键组件， 即模**型构建**、**参数访问与初始化**、**设计自定义层和块**、**将模型读写到磁盘**， 以及**利用GPU实现显著的加速**。 这些知识将使你从深度学习“基础用户”变为“高级用户”。 虽然本章不介绍任何新的模型或数据集， 但后面的高级模型章节在很大程度上依赖于本章的知识。

# 5.1. 层和块
- 对于多层感知机而言，整个模型接受原始输入（特征），生成输出（预测）， 并包含一些参数（所有组成层的参数集合）。 同样，每个单独的层接收输入（由前一层提供）， 生成输出（到下一层的输入），并且具有一组可调参数， 这些参数根据从下一层反向传播的信号进行更新。
- 为了实现这些复杂的网络，我们引入了**神经网络块**的概念。 块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 使用块进行抽象的一个好处是可以将一些块组合成更大的组件，这一过程通常是递归的，如图所示。 通过定义代码来按需生成任意复杂度的块，我们可以通过简洁的代码实现复杂的神经网络。

![avatar](../img/5_1.png)

- 从编程的角度来看，**块由类（class）表示**。 它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 最后，为了计算梯度，**块必须具有反向传播函数**。 在定义我们自己的块时，由于自动微分（在 2.5节中引入） 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

- 在构造自定义块之前，我们先回顾一下多层感知机（4.3节）的代码。

In [97]:
import torch
from torch import nn
from torch.nn import functional as F

"""
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层， 然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。
"""

"""
在这个例子中，我们通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。

简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。

注意，两个全连接层都是Linear类的实例， Linear类本身就是Module的子类。 另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。 这实际上是net.__call__(X)的简写。

这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。
"""

net = nn.Sequential(nn.Linear(20,256),
                    nn.ReLU(),
                    nn.Linear(256,10))

# Returns a tensor filled with random numbers from a uniform distribution on the interval :math:`[0, 1)`
X = torch.rand(2,20)
net(X)

tensor([[ 0.2206, -0.0271,  0.0491, -0.0983,  0.1422, -0.3221,  0.3151, -0.0525,
         -0.1037,  0.3866],
        [ 0.1604,  0.1691,  0.0348, -0.1417, -0.0765, -0.2648,  0.2983,  0.0658,
          0.0056,  0.3420]], grad_fn=<AddmmBackward0>)

## 5.1.1. 自定义块
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能：
1. 将输入数据作为其前向传播函数的参数。
2. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
4. 存储和访问前向传播计算所需的参数。
5. 根据需要初始化模型参数。

In [98]:
"""
在下面的代码片段中，我们从零开始编写一个块。 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

我们首先看一下前向传播函数，它以X作为输入， 计算带有激活函数的隐藏表示，并输出其未规范化的输出值。 在这个MLP实现中，两个层都是实例变量。 要了解这为什么是合理的，可以想象实例化两个多层感知机（net1和net2）， 并根据不同的数据对它们进行训练。 当然，我们希望它们学到两种不同的模型。

接着我们实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层。 注意一些关键细节： 首先，我们定制的__init__函数通过super().__init__() 调用父类的__init__函数， 省去了重复编写模版代码的痛苦。 然后，我们实例化两个全连接层， 分别为self.hidden和self.out。 注意，除非我们实现一个新的运算符， 否则我们不必担心反向传播函数或参数初始化， 系统将自动生成这些。
"""
class MLP(nn.Module):

    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden=nn.Linear(20,256)  # 隐藏层
        self.out=nn.Linear(256,10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))


net=MLP()
net(X)

tensor([[ 0.0108, -0.1091, -0.0496,  0.1179, -0.0973, -0.0829, -0.1096,  0.0240,
         -0.2941,  0.0065],
        [-0.1016, -0.0169, -0.1666,  0.0252, -0.0775, -0.0105, -0.2196,  0.0149,
         -0.2174, -0.0096]], grad_fn=<AddmmBackward0>)

## 5.1.2. 顺序块
现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：
1. 一种将块逐个追加到列表中的函数。
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [100]:
"""
下面的MySequential类提供了与默认Sequential类相同的功能。

__init__函数将每个模块逐个添加到有序字典_modules中。 你可能会好奇为什么每个Module都有一个_modules属性？ 以及为什么我们使用它而不是自己定义一个Python列表？

简而言之，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。
"""
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in  enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)]=module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X=block(X)
        return X

"""
当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 现在可以使用我们的MySequential类重新实现多层感知机。
请注意，MySequential的用法与之前为Sequential类编写的代码相同
"""
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0453,  0.1498,  0.0397, -0.1982, -0.0777,  0.0050,  0.0254,  0.1487,
         -0.1930, -0.0588],
        [-0.0810,  0.0568, -0.0478, -0.2261, -0.0936, -0.0136, -0.0380,  0.0755,
         -0.1104, -0.1415]], grad_fn=<AddmmBackward0>)

## 5.1.3. 在前向传播函数中执行代码
Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，我们称之为**常数参数**（constant parameter）。例如，我们需要一个计算函数$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，其中$\mathbf{x}$是输入，$\mathbf{w}$是参数，$c$是某个在优化过程中没有更新的指定常量。

因此我们实现了一个`FixedHiddenMLP`类，如下所示：

In [113]:
"""
在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在L1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止。 最后，模型返回了X中所有项的和。 注意，此操作可能不会常用于在任何实际任务中， 我们只是向你展示如何将任意代码集成到神经网络计算的流程中。
"""
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 该对象没有梯度，不参与训练
        self.rand_weight=torch.rand((20,20),requires_grad=False)

        self.linear=nn.Linear(20,20)

    def forward(self,X):
        X=self.linear(X)

        # 手写一个操作torch.mm
        X=F.relu(torch.mm(X,self.rand_weight)+1)

        X=self.linear(X)

        # 若X的绝对值求和大于1
        while X.abs().sum() >1:
            X /= 2
        return X.sum()

net=FixedHiddenMLP()
net(X)  # 得到一个标量

tensor(-0.0186, grad_fn=<SumBackward0>)

In [121]:
"""
我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。
"""
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))


chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.1903, grad_fn=<SumBackward0>)

## 5.1.4. 效率
你可能会开始担心操作效率的问题。 毕竟，我们在一个高性能的深度学习库中进行了大量的字典查找、 代码执行和许多其他的Python代码。 Python的问题**全局解释器锁**是众所周知的。 在深度学习环境中，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。

### global interpreter lock（GIL）
- It is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecodes at once.
- The GIL prevents race conditions and ensures thread safety.

## 5.1.5. 小结
- 一个块可以由许多层组成；一个块可以由许多块组成。
- 块可以包含代码。
- 块负责大量的内部处理，包括参数初始化和反向传播。
- 层和块的顺序连接由Sequential块处理。